======= 1. requirements matching  > 0.85
            1. semantic analysis > 0.9
                a. similarity keywords
            2. Nace Code match > 1.0
        2. Location matching
            1. exact location matching
            2. Geocoding matching 

In [1]:
import json
import pandas as pd
import spacy
import nltk
import numpy as np
import re
import shelve
import json
import os
import logging
import gc
import torch

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, CrossEncoder
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
from geopy.distance import geodesic 

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [3]:
# DATA IMPORTS
dataDIR = './data/'
originalSalesNexxtChangeData = f'{dataDIR}branche_nexxt_change_sales_listings_scrape.csv'
dejunaPurchases = './data/dejuna_buyer_latest.csv'

nacecode_josn =  './data/nace_codes.json' 
nacecode_array_josn =  './data/nace_codes_array.json' 
nacecode_array_obj =  './data/nace_codes_object.json' 
nacecode_array_obj_ext =  './data/nace_codes_object_ext.json' 
nacecode_array_obj_du =  './data/nace_codes_object_du.json'
 
dataFile =  './data/nexxt_change_sales_listings_geocoded_short_test.csv' 
# sales_file_nace =  './data/nexxt_change_sales_listings_geocoded.csv' 
sales_file_brachen =  './data/branche_nexxt_change_sales_listings.csv' 
sales_file_nace =  './data/dub_listings_geo.csv'
buyer_file_nace =  './data/nexxt_change_purchase_listings_geocoded.csv' 

In [4]:
# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')

# -------------------------------------------------------------------------
# 1. Load SpaCy's German model (for tokenization, NER, POS tagging)
# -------------------------------------------------------------------------
try:
    nlp = spacy.load('de_core_news_sm')
except OSError:
    from spacy.cli import download
    download('de_core_news_sm')
    nlp = spacy.load('de_core_news_sm')

[nltk_data] Downloading package punkt to /Users/abbasm1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abbasm1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# -------------------------------------------------------------------------
# 2. Preprocessing function (with German NER, POS, etc.)
# -------------------------------------------------------------------------
def preprocess_text(text, nlp_model):
    """
    - Lowercases the text.
    - Removes URLs, emails, & large digit sequences.
    - Filters out non-alphabetic chars except German Umlauts/ß.
    - Uses SpaCy to keep only NOUN, PROPN, VERB tokens not in stopwords.
    - Applies Snowball stemming on remaining tokens.
    - Also includes certain named entities (ORG, PRODUCT, GPE).
    """
    if pd.isnull(text):
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs, emails, large numbers
    text = re.sub(r'http\S+|www.\S+|\S+@\S+|\b\d{10,}\b', '', text)
    
    # Keep only letters and German characters
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    
    # Compact multiple spaces
    text = ' '.join(text.split())

    # Initialize German stopwords and Snowball stemmer
    stop_words = set(stopwords.words('german'))
    stemmer = SnowballStemmer('german')

    # Process text with SpaCy
    doc = nlp_model(text)

    tokens = []
    for token in doc:
        # Keep nouns, proper nouns, and verbs
        if token.pos_ in {'NOUN', 'PROPN', 'VERB'} and token.text not in stop_words:
            stemmed = stemmer.stem(token.text)
            tokens.append(stemmed)

    # Extract named entities (ORG, PRODUCT, GPE) and include them
    entities = [
        ent.text for ent in doc.ents 
        if ent.label_ in {'ORG', 'PRODUCT', 'GPE'}
    ]
    # Stem and remove stopwords from entities
    entities = [
        stemmer.stem(ent.lower()) 
        for ent in entities 
        if ent.lower() not in stop_words
    ]
    tokens.extend(entities)

    return ' '.join(tokens)

In [6]:
def _extract_location_parts(location):
    """Extract and categorize location parts into states and cities."""
    locations = set()
    german_states = {
        'baden-württemberg', 'bayern', 'berlin', 'brandenburg', 'bremen',
        'hamburg', 'hessen', 'mecklenburg-vorpommern', 'niedersachsen',
        'nordrhein-westfalen', 'rheinland-pfalz', 'saarland', 'sachsen',
        'sachsen-anhalt', 'schleswig-holstein', 'thüringen'
    }


    if not location or not isinstance(location, str):
        return locations

    try:
        # Split on common delimiters
        parts = re.split(r'[>/\n]\s*', location)
        split_locations = []

        for part in parts:
            part = part.strip().lower()
            if part:
                # Further split by space if multiple states are concatenated
                words = part.split()
                temp = []
                current = ""
                for word in words:
                    if word.lower() in german_states:
                        if current:
                            temp.append(current.strip())
                        current = word
                    else:
                        current += " " + word if current else word
                if current:
                    temp.append(current.strip())
                split_locations.extend(temp)

        for loc in split_locations:
            loc = loc.strip().lower()
            if loc:
                if loc in german_states:
                    locations.add(loc.title())  # Capitalize for better geocoding
                else:
                    # Clean up common prefixes like "region"
                    clean_part = re.sub(r'^region\s+', '', loc)
                    if clean_part:
                        locations.add(clean_part.title())
    except Exception as e:
        logging.error(f"Error extracting location parts: {e}")

    return locations


# def _extract_location_parts(location):
#     """Extract and categorize location parts into states, districts, or cities."""
#     locations = set()

#     if not location or not isinstance(location, str):
#         return locations

#     try:
#         # Split location string by " > ", handling the hierarchical structure
#         parts = re.split(r'[\n]\s*', location)

#         for part in parts:
#             part = part.split(">")
#             locations.add(part[-1].strip())

#     except Exception as e:
#         print(f"Error extracting location parts: {e}")

#     return list(locations)


In [7]:
# _extract_location_parts('''Sachsen / Leipzig / Leipzig, Stadt''')
_extract_location_parts('''Berlin
Sachsen > Leipzig
Sachsen > Dresden
Brandenburg
Niedersachsen > Hannover
Hessen > Frankfurt am Main
                        Hamburg
                        Bayern > München
                        Bayern > Nürnberg
                        Bayern > Augsburg
                        A
                        B > C''')

{'A',
 'Augsburg',
 'B',
 'Bayern',
 'Berlin',
 'Brandenburg',
 'C',
 'Dresden',
 'Frankfurt Am Main',
 'Hamburg',
 'Hannover',
 'Hessen',
 'Leipzig',
 'München',
 'Niedersachsen',
 'Nürnberg',
 'Sachsen'}

NACE CODE

In [8]:
# -------------------------------------------------------------------------
# 3. Load NACE codes from JSON
# -------------------------------------------------------------------------
def load_nace_codes(filepath):
    """
    Expects a JSON file where keys = NACE code, values = textual descriptions.
    Example:
      {
        "01.1": "Growing of non-perennial crops",
        "01.2": "Growing of perennial crops",
        ...
      }
    """
    with open(filepath, 'r', encoding='utf-8') as file:
        nace_codes = json.load(file)
    return nace_codes

In [9]:
# -------------------------------------------------------------------------
# 4. Create embeddings with sentence-transformers
# -------------------------------------------------------------------------
def get_embedding_batch(texts, model, batch_size=64):
    """
    Encode texts in batches to optimize memory usage.
    """
    embeddings = model.encode(texts, batch_size=batch_size, show_progress_bar=True, 
                              convert_to_numpy=True, normalize_embeddings=True)
    return embeddings.astype('float32')  # Use float32 to save memory

In [ ]:

# -------------------------------------------------------------------------
# 5. Load your Seller/Branchen data and NACE codes
# -------------------------------------------------------------------------
def load_data(nace_codes_filepath):
    nace_codes = load_nace_codes(nace_codes_filepath)
    return nace_codes

# -------------------------------------------------------------------------
# 6. MAIN LOGIC
# -------------------------------------------------------------------------
    # Filepaths (update to your actual paths)
# sellers_filepath = originalSalesNexxtChangeData       # CSV with a column 'branchen'
sellers_filepath =  dejunaPurchases    # CSV with a column 'branchen'
nace_codes_filepath = nacecode_array_obj_du

# Load data
sellers_df = pd.read_csv(sellers_filepath) 
nace_codes = load_data(nace_codes_filepath)
print("🚀 Sellers and NACE codes loaded.")

# Initialize the Sentence Transformer model
# For German or multilingual, consider e.g.: 
#    model_name = 'paraphrase-multilingual-mpnet-base-v2'
model_name = 'all-MiniLM-L12-v2'
model = SentenceTransformer(model_name)
print(f"🚀 Loaded SentenceTransformer model: {model_name}")

# ---------------------------------------------------------------------
# 6a. Preprocess NACE descriptions
# ---------------------------------------------------------------------
# Convert each NACE code's description with the same text preprocessing
nace_descriptions = [preprocess_text(desc, nlp) for desc in nace_codes.values()]
# Create embeddings for these descriptions
nace_embeddings = get_embedding_batch(nace_descriptions, model)
# We'll keep a list of NACE codes in the same order
nace_code_list = list(nace_codes.keys())
print("🚀 Created embeddings for NACE descriptions.")

# ---------------------------------------------------------------------
# 6b. Preprocess 'branchen' column in sellers data
# ---------------------------------------------------------------------
# We'll store it in a new column 'preprocessed_branchen'
# sellers_df['preprocessed_branchen'] = sellers_df['branchen'].apply(lambda x: preprocess_text(x, nlp))
# Split 'branchen' on '>' and join each part


logging.info('Preprocessing buyers\' text fields...')
sellers_df['title_preprocessed'] = sellers_df['title'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['description_preprocessed'] = sellers_df['description'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['long_description_preprocessed'] = sellers_df['long_description'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['preprocessed_branchen'] = sellers_df.apply(
    lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
)

# sellers_df['preprocessed_branchen'] = sellers_df['branchen'].apply(
#     lambda x: ' '.join(x.split('>'))
# )

def combine_text_fields(row):
    return ' '.join([
        row.get('title_preprocessed', ''),
        row.get('description_preprocessed', ''),
        row.get('long_description_preprocessed', ''),
        row.get('branchen_preprocessed', '')
    ])
sellers_df['combined_text'] = sellers_df.apply(combine_text_fields, axis=1)

# # Join 'Sub-Industrie' and 'Industrie' columns for buyer data
# sellers_df['preprocessed_branchen'] = sellers_df.apply(
#     lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
# )

# Create embeddings for all sellers' branchen
branchen_embeddings = get_embedding_batch(sellers_df['combined_text'].tolist(), model)
print("🚀 Created embeddings for sellers' 'branchen' field.")

# ---------------------------------------------------------------------
# 6c. Compute similarity and assign best-match NACE code
# ---------------------------------------------------------------------
similarities = cosine_similarity(branchen_embeddings, nace_embeddings)
best_match_indices = similarities.argmax(axis=1)

# For each seller row, pick the NACE code with highest similarity
sellers_df['assigned_nace_code'] = [nace_code_list[idx] for idx in best_match_indices]
sellers_df['assigned_nace_similarity'] = [similarities[i][idx] for i, idx in enumerate(best_match_indices)]
print("🚀 Assigned preliminary NACE codes based on 'branchen' similarity.")

# Optionally set a threshold. If similarity < threshold => 'Unassigned'
similarity_threshold = 0.4
sellers_df['nace_code'] = sellers_df.apply(
    lambda row: row['assigned_nace_code'] 
                if row['assigned_nace_similarity'] >= similarity_threshold 
                else 'Unassigned',
    axis=1
)

# ---------------------------------------------------------------------
# 6d. Save and review
# ---------------------------------------------------------------------
output_file = sellers_filepath.replace(".csv", "_nace.csv")
sellers_df.to_csv(output_file, index=False)
print(f"🚀 Saved sellers data with assigned NACE codes to: {output_file}\n")

# Print a small sample
print("Sample of NACE Code Assignments:")
print(sellers_df[['assigned_nace_code', 'assigned_nace_similarity', 'nace_code']].head(10))


Another NACE code

In [ ]:

# -------------------------------------------------------------------------
# 4. Create embeddings using Hugging Face
# -------------------------------------------------------------------------
def create_hf_embeddings(texts, tokenizer, model):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
            embeddings.append(cls_embedding.squeeze().numpy())
    return np.vstack(embeddings)

# -------------------------------------------------------------------------
# 5. Load Sellers and NACE Data
# -------------------------------------------------------------------------
def load_data(sellers_filepath, nace_codes_filepath):
    sellers_df = pd.read_csv(sellers_filepath)
    nace_codes = load_nace_codes(nace_codes_filepath)
    return sellers_df, nace_codes

# -------------------------------------------------------------------------
# 6. MAIN LOGIC
# -------------------------------------------------------------------------
# Filepaths (update to your actual paths)
# sellers_filepath = './data/dejuna_buyer_latest.csv'
sellers_filepath = originalSalesNexxtChangeData
nace_codes_filepath = './data/nace_codes_object_du.json'

# Load data
sellers_df, nace_codes = load_data(sellers_filepath, nace_codes_filepath)
print("🚀 Sellers and NACE codes loaded.")

# Initialize Hugging Face model and tokenizer
# model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
print(f"🚀 Loaded Hugging Face model: {model_name}")

# ---------------------------------------------------------------------
# 6a. Preprocess NACE descriptions
# ---------------------------------------------------------------------
nace_descriptions = [preprocess_text(desc, nlp) for desc in nace_codes.values()]
nace_embeddings = create_hf_embeddings(nace_descriptions, tokenizer, model)
nace_code_list = list(nace_codes.keys())
print("🚀 Created embeddings for NACE descriptions.")

# ---------------------------------------------------------------------
# 6b. Preprocess 'branchen' column in sellers data
# ---------------------------------------------------------------------

# sellers_df['preprocessed_branchen'] = sellers_df.apply(
#  lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
# )
sellers_df['preprocessed_branchen'] = sellers_df['branchen'].apply(lambda x: preprocess_text(x, nlp))
branchen_embeddings = create_hf_embeddings(sellers_df['preprocessed_branchen'].tolist(), tokenizer, model)
print("🚀 Created embeddings for sellers' 'branchen' field.")

# ---------------------------------------------------------------------
# 6c. Compute similarity and assign best-match NACE code
# ---------------------------------------------------------------------
similarities = cosine_similarity(branchen_embeddings, nace_embeddings)
best_match_indices = similarities.argmax(axis=1)

sellers_df['assigned_nace_code'] = [nace_code_list[idx] for idx in best_match_indices]
sellers_df['assigned_nace_similarity'] = [similarities[i][idx] for i, idx in enumerate(best_match_indices)]

similarity_threshold = 0.7
sellers_df['nace_code'] = sellers_df.apply(
    lambda row: row['assigned_nace_code'] if row['assigned_nace_similarity'] >= similarity_threshold else 'Unassigned',
    axis=1
)

# ---------------------------------------------------------------------
# 6d. Save and review
# ---------------------------------------------------------------------
output_file = sellers_filepath.replace(".csv", "_hf_nace.csv")
sellers_df.to_csv(output_file, index=False)
print(f"🚀 Saved sellers data with assigned NACE codes to: {output_file}\n")

print("Sample of NACE Code Assignments:")
print(sellers_df[[ 'assigned_nace_code', 'assigned_nace_similarity', 'nace_code']].head(10))


GEOCODING LOCATION

In [82]:
def get_all_unique_locations(buyers_df, sellers_df):
    """Extract all unique locations from buyers and sellers dataframes."""
    unique_locations = set()

    for df, name in [(buyers_df, 'buyers'), (sellers_df, 'sellers')]:
        logging.info(f'Extracting locations from {name} dataframe...')
        for idx, location in df['location'].items():
            locations = _extract_location_parts(location)
            
            unique_locations.update(locations)

    logging.info(f'Total unique locations found: {len(unique_locations)}')
    return unique_locations

def geocode_locations(unique_locations, cache_path='geocode_cache.db'):
    """Geocode unique locations with caching."""
    geolocator = Nominatim(user_agent="buyer_seller_matching")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=10.0)

    # Ensure cache directory exists
    cache_dir = os.path.dirname(cache_path)
    if cache_dir and not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    with shelve.open(cache_path) as geocode_cache:
        for location in unique_locations:

            if location in geocode_cache:
                continue  # Already cached
            try:
                logging.info(f'Geocoding location: {location}')
                loc = geocode(location + ", Germany")
                if loc:
                    geocode_cache[location] = {'latitude': loc.latitude, 'longitude': loc.longitude}
                    logging.info(f'Geocoded {location}: ({loc.latitude}, {loc.longitude})')
                else:
                    geocode_cache[location] = {'latitude': None, 'longitude': None}
                    logging.warning(f'Geocoding failed for location: {location}')
            except Exception as e:
                logging.error(f"Geocoding error for location '{location}': {e}")
                geocode_cache[location] = {'latitude': None, 'longitude': None}

def update_dataframe_with_geocodes(df, cache_path='geocode_cache.db'):
    """Add latitude and longitude columns to the dataframe based on locations."""
    with shelve.open(cache_path) as geocode_cache:
        latitudes = []
        longitudes = []

        for idx, location in df['location'].items():
            locations = _extract_location_parts(location)
            lat_list = []
            lon_list = []
            for loc in locations:
                geocode_info = geocode_cache.get(loc, {'latitude': None, 'longitude': None})
                if geocode_info['latitude'] is not None and geocode_info['longitude'] is not None:
                    lat_list.append(geocode_info['latitude'])
                    lon_list.append(geocode_info['longitude'])
                else:
                    # If geocoding failed, append None
                    lat_list.append(None)
                    lon_list.append(None)
            # Convert lists to JSON strings for CSV compatibility
            latitudes.append(json.dumps(lat_list))
            longitudes.append(json.dumps(lon_list))

    df['latitude'] = latitudes
    df['longitude'] = longitudes
    return df

# Paths to input and output files
buyer_filepath = './data/dejuna_buyer_latest_hf_nace.csv'
sellers_filepath = './data/branche_nexxt_change_sales_listings_scrape_hf_nace.csv'

cache_path = './geocode_cache.db'

# Load buyer and seller datasets

sellers_df = pd.read_csv(sellers_filepath)
buyers_df = pd.read_csv(buyer_filepath)
unique_locations = get_all_unique_locations(buyers_df, sellers_df)
# Geocode locations with caching
geocode_locations(unique_locations, cache_path=cache_path)

# Update dataframes with geocodes

# logging.info('Updating sellers dataframe with geocodes...')
sellers_df = update_dataframe_with_geocodes(sellers_df, cache_path=cache_path)
buyers_df = update_dataframe_with_geocodes(buyers_df, cache_path=cache_path)

# # Save updated dataframes to new CSV files
logging.info('Saving updated sellers dataframe...')
sellers_output_file = sellers_filepath.replace(".csv", "_geocoded.csv")
buyers_output_file = buyer_filepath.replace(".csv", "_geocoded.csv")
sellers_df.to_csv(sellers_output_file, index=False)
buyers_df.to_csv(buyers_output_file, index=False)
logging.info('Geocoding process completed successfully.')
print(f"🚀 Saved sellers data with assigned geocodes to: {buyers_output_file}\n {sellers_output_file}\n")



2025-01-19 20:58:36,799 - INFO - Extracting locations from buyers dataframe...
2025-01-19 20:58:36,800 - INFO - Extracting locations from sellers dataframe...
2025-01-19 20:58:36,805 - INFO - Total unique locations found: 419
2025-01-19 20:58:36,834 - INFO - Geocoding location: Freiburg im Breisgau
2025-01-19 20:58:37,164 - INFO - Geocoded Freiburg im Breisgau: (47.9960901, 7.8494005)
2025-01-19 20:58:37,167 - INFO - Geocoding location: Landsberg am Lech
2025-01-19 20:58:38,263 - INFO - Geocoded Landsberg am Lech: (48.0497474, 10.8768728)
2025-01-19 20:58:38,265 - INFO - Geocoding location: Offenbach am Main
2025-01-19 20:58:39,186 - INFO - Geocoded Offenbach am Main: (50.1055002, 8.7610698)
2025-01-19 20:58:39,187 - INFO - Geocoding location: Rüdesheim am Rhein
2025-01-19 20:58:40,208 - INFO - Geocoded Rüdesheim am Rhein: (49.9789358, 7.92339)
2025-01-19 20:58:40,209 - INFO - Geocoding location: Weiden i.d.OPf.
2025-01-19 20:58:41,163 - INFO - Geocoded Weiden i.d.OPf.: (49.6752749, 12

🚀 Saved sellers data with assigned geocodes to: ./data/dejuna_buyer_latest_hf_nace_geocoded.csv
 ./data/branche_nexxt_change_sales_listings_scrape_hf_nace_geocoded.csv



In [68]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

def geocode(location):
    """
    Geocodes a location string to latitude and longitude.
    
    Args:
        location (str): The location string to geocode.
    
    Returns:
        tuple: Latitude and longitude of the location, or None if not found.
    """
    geolocator = Nominatim(user_agent="geoapi")
    location_data = geolocator.geocode(location)
    if location_data:
        return location_data.latitude, location_data.longitude
    else:
        return None

# Example usage
location1 = "Sachsen"
location2 = "Ausland"
#  {'Berlin',
#  'Brandenburg',
#  'Dresden',
#  'Hannover',
#  'Leipzig',
#  'Niedersachsen',
#  'Sachsen'}
coordinates1 = geocode(location1 + ", Germany")
coordinates2 = geocode(location2 + ", Germany")

if coordinates1 and coordinates2:
    distance = geodesic(coordinates1, coordinates2).kilometers
    print(f"Coordinates for '{location1}': {coordinates1}")
    print(f"Coordinates for '{location2}': {coordinates2}")
    print(f"Distance between '{location1}' and '{location2}': {distance:.2f} km")
else:
    print("One or both locations could not be geocoded.")


2025-01-19 12:19:59,937 - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Ausland%2C+Germany&format=json&limit=1


Coordinates for 'Sachsen': (50.9295798, 13.4585052)
Coordinates for 'Ausland': (52.5448387, 13.4194785)
Distance between 'Sachsen' and 'Ausland': 179.74 km


In [59]:
# sales[sales['standort'].apply(lambda x: '''Berlin
# Sachsen > Leipzig
# Sachsen > Dresden
# Brandenburg
# Niedersachsen > Hannover''' in x)];

purchase[purchase['location'].apply(lambda x: '''Berlin
Sachsen > Leipzig
Sachsen > Dresden
Brandenburg
Niedersachsen > Hannover''' in x)]

,date,location,title,description,long_description,source,contact details,Industrie,Sub-Industrie,preprocessed_branchen,assigned_nace_code,assigned_nace_similarity,nace_code,latitude,longitude,processed_location
12,28.03.24,Berlin\nSachsen > Leipzig\nSachsen > Dresden\n...,Geschäftsführer sucht Dienstleistungsunternehmen,Christian ist Geschäftsführer im Logistik-Bere...,Gesucht wird ein etabliertes und profitables D...,dejuna,Dr. Christian Schneider\nDeine E-Mail: christi...,Transport und Logistik,"Logistikdienstleistungen, Spedition",transport logist logistikdienstleist spedition,H52.2.1,0.797184,H52.2.1,"[52.8455492, 52.510885, 51.3406321, 52.3744779...","[13.2461296, 13.3989367, 12.3747329, 9.7385532...","[Brandenburg, Berlin, Leipzig, Hannover, Niede..."


Location Matching

In [ ]:

sales = pd.read_csv('./data/branche_nexxt_change_sales_listings_scrape_hf_nace_geocoded.csv')[10:30]
purchase = pd.read_csv('./data/dejuna_buyer_latest_hf_nace_geocoded.csv')[10:30]


sales['processed_location'] = sales['location'].apply(lambda x: list(_extract_location_parts(x)))
purchase['processed_location'] = purchase['location'].apply(lambda x: list(_extract_location_parts(x)))

# # Check if any element of sales processed_location is in purchase processed_location
purchase['latitude'] = purchase['latitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])
purchase['longitude'] = purchase['longitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])

sales['latitude'] = sales['latitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])
sales['longitude'] = sales['longitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])


def match_locations(sales_locations, purchase_locations):
    return any(loc in purchase_locations for loc in sales_locations)

# Create a dataframe to store matched locations
matched_locations = []

for idx, row in sales.iterrows():
    for idx2, row2 in purchase.iterrows():
        if match_locations(row.processed_location, row2.processed_location):
            matched_locations.append({
                'sales_id': idx,
                'sales_location': row.location,
                'sales_processed_location': row.processed_location,
                'purchase_id': idx2,
                'purchase_location': row2.location,
                'sales_longitude': s_lon,
                'sales_latitude': s_lat,
                'purchase_longitude': p_lon,
                'purchase_latitude': p_lat,
                'purchase_processed_location': row2.processed_location,
                'distance_km': None
            })
        else:
            # Calculate distance between sales and purchase locations
            sales_lat_lon = zip(row.latitude, row.longitude)
            purchase_lat_lon = zip(row2.latitude, row2.longitude)
            for s_lat, s_lon in sales_lat_lon:
                for p_lat, p_lon in purchase_lat_lon:
                    if s_lat is not None and s_lon is not None and p_lat is not None and p_lon is not None:
                        print(f"Calculating distance between ({s_lat}, {s_lon}) and ({p_lat}, {p_lon})")
                        distance = geodesic((s_lat, s_lon), (p_lat, p_lon)).km
                        print(f"Distance: {distance}")
                        if distance <= 50:
                            matched_locations.append({
                                'sales_id': idx,
                                'sales_processed_location': row.processed_location,
                                'sales_location': row.location,
                                'sales_longitude': s_lon,
                                'sales_latitude': s_lat,
                                'purchase_longitude': p_lon,
                                'purchase_latitude': p_lat,
                                'purchase_id': idx2,
                                'purchase_location': row2.location,
                                'purchase_processed_location': row2.processed_location,
                                'distance_km': distance
                            })
matched_locations_df = pd.DataFrame(matched_locations)
print(matched_locations_df)


Semantic Analysis

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
german_stop_words = stopwords.words('german')

# Load synonyms CSV
synonyms_df = pd.read_csv('./data/Updated_Keywords_and_Synonyms.csv')
synonym_dict = {}
for _, row in synonyms_df.iterrows():
    keyword = row['Keyword'].lower()
    synonyms = row.dropna().tolist()[1:]
    synonyms = [syn.lower() for syn in synonyms]
    synonym_dict[keyword] = synonyms

# Define augmentation functions
def extract_keywords(text, top_n=5):
    # vectorizer = TfidfVectorizer(stop_words='german', max_features=top_n)
    vectorizer = CountVectorizer(stop_words = german_stop_words) # Now use this in your pipeline

    tfidf_matrix = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out()

def augment_text_with_synonyms(text, top_n=5):
    keywords = extract_keywords(text, top_n)
    synonyms = []
    for word in keywords:
        synonyms.extend(synonym_dict.get(word, []))
    synonyms = ' '.join(synonyms)
    return f"{text} {synonyms}"

# Initialize spaCy
nlp_de = spacy.load('de_core_news_sm')

def preprocess_text_de(text):
    doc = nlp_de(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)


def combine_text_fields(row):
    return ' '.join([
        row.get('title_preprocessed', ''),
        row.get('description_preprocessed', ''),
        row.get('long_description_preprocessed', ''),
        row.get('branchen_preprocessed', '')
    ])

def analyze_matches(matches_df, buyers_df, sellers_df):
    """Analyze matching results and print key metrics."""
    logging.info("\n=== Matching Analysis ===")
    
    total_buyers = len(buyers_df)
    total_sellers = len(sellers_df)
    total_matches = len(matches_df)
    
    logging.info(f"Total buyers: {total_buyers}")
    logging.info(f"Total sellers: {total_sellers}") 
    logging.info(f"Total matches found: {total_matches}")
    if total_buyers > 0:
        logging.info(f"Average matches per buyer: {total_matches/total_buyers:.2f}")
    else:
        logging.info("No buyers to match against.")

    # Save top matches for manual review
    top_matches = matches_df.head(10)
    top_matches.to_csv('./matches/top_matches_for_review.csv', index=False)
    logging.info("Saved top 10 matches for manual review")
    
    return {
        'total_matches': total_matches,
        'matches_per_buyer': total_matches/total_buyers if total_buyers else 0,
        'buyer_match_rate': len(matches_df['buyer_title'].unique())/total_buyers if total_buyers else 0
    }


# Load updated datasets
logging.info('Loading datasets...')
buyers_df = pd.read_csv('./data/dejuna_buyer_latest_hf_nace.csv')
sellers_df = pd.read_csv('./data/branche_nexxt_change_sales_listings_scrape_hf_nace.csv')

# Preprocess buyers' text fields
logging.info('Preprocessing buyers\' text fields...')
buyers_df['title_preprocessed'] = buyers_df['title'].apply(lambda x:  preprocess_text(x, nlp))
buyers_df['description_preprocessed'] = buyers_df['description'].apply(lambda x:  preprocess_text(x, nlp))
buyers_df['long_description_preprocessed'] = buyers_df['long_description'].apply(lambda x:  preprocess_text(x, nlp))
buyers_df['preprocessed_branchen'] = buyers_df.apply(
    lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
)
# Preprocess sellers' text fields
logging.info('Preprocessing sellers\' text fields...')
sellers_df['title_preprocessed'] = sellers_df['title'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['description_preprocessed'] = sellers_df['description'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['long_description_preprocessed'] = sellers_df['long_description'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['branchen_preprocessed'] = sellers_df['branchen'].apply(lambda x:  preprocess_text(x, nlp))

# Combine text fields
logging.info('Combining text fields...')
buyers_df['combined_text'] = buyers_df.apply(combine_text_fields, axis=1)
sellers_df['combined_text'] = sellers_df.apply(combine_text_fields, axis=1)


# Augment text with synonyms
# logging.info('Augmenting buyers\' text with synonyms...')
# buyers_df['augmented_text'] = buyers_df['combined_text'].apply(lambda x: augment_text_with_synonyms(x, top_n=5))
# logging.info('Augmenting sellers\' text with synonyms...')
# sellers_df['augmented_text'] = sellers_df['combined_text'].apply(lambda x: augment_text_with_synonyms(x, top_n=5))

# # Preprocess augmented text
# buyers_df['final_text'] = buyers_df['combined_text'].apply(preprocess_text_de) + ' ' + buyers_df['augmented_text'].apply(preprocess_text_de)
# sellers_df['final_text'] = sellers_df['combined_text'].apply(preprocess_text_de) + ' ' + sellers_df['augmented_text'].apply(preprocess_text_de)


# Load the Sentence Transformer model
logging.info('Loading the Sentence Transformer model...')
# model_name = 'paraphrase-multilingual-mpnet-base-v2'
model_name = 'all-MiniLM-L12-v2'
# model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

try:
    model = SentenceTransformer(model_name)
except Exception as e:
    logging.error(f"Error loading model {model_name}: {e}")

# Encode sellers' combined_text
logging.info("Encoding sellers' text...")
seller_texts = sellers_df['combined_text'].tolist()
seller_embeddings = get_embedding_batch(seller_texts, model, batch_size=64)
logging.info("Sellers' embeddings generated.")

# Set similarity threshold
similarity_threshold = 0.78
# Optional text length filter

matches = []
confidence_scores = []

total_buyers = len(buyers_df)
logging.info('Starting matching process...')
for i, buyer_row in buyers_df.iterrows():
    buyer_text = buyer_row['combined_text']
    

    # Encode buyer text
    buyer_embedding = model.encode(buyer_text, convert_to_numpy=True, normalize_embeddings=True).reshape(1, -1)

    # Calculate similarity scores to all sellers
    sim_scores = cosine_similarity(buyer_embedding, seller_embeddings)[0]

    # Indices above threshold
    matching_indices = np.where(sim_scores >= similarity_threshold)[0]

    for seller_idx in matching_indices:
        seller_row = sellers_df.iloc[seller_idx]

        confidence_score = sim_scores[seller_idx]
        if confidence_score < similarity_threshold:
            continue

        confidence_scores.append(confidence_score)

        match = {
            'buyer_date': buyer_row.get('date', ''),
            'buyer_title': buyer_row.get('title', ''),
            'buyer_description': buyer_row.get('description', ''),
            'buyer_long_description': buyer_row.get('long_description', ''),
            'buyer_location': buyer_row.get('location', ''),
            'buyer_nace_code': buyer_row.get('nace_code', ''), 

            'seller_date': seller_row.get('date', ''),
            'seller_title': seller_row.get('title', ''),
            'seller_description': seller_row.get('description', ''),
            'seller_long_description': seller_row.get('long_description', ''),
            'seller_location': seller_row.get('location', ''),
            'seller_nace_code': seller_row.get('nace_code', ''),
            
            'similarity_score': confidence_score
        }
        matches.append(match)

    # Progress logging
    if (i + 1) % 50 == 0:
        logging.info(f"Processed {i+1}/{total_buyers} buyers.")

logging.info('Creating matches DataFrame...')
matches_df = pd.DataFrame(matches)

if not matches_df.empty:
    matches_df['confidence_score'] = confidence_scores

    # Sort by confidence score
    matches_df = matches_df.sort_values('confidence_score', ascending=False)

    # Save all matches
    timestamp = datetime.now().strftime("%d_%H-%M")
    output_all = f'./matches/nlp_business_all_matches_{timestamp}.csv'
    matches_df.to_csv(output_all, index=False)
    logging.info(f'Saved all matches: {len(matches_df)} records => {output_all}')

    # Analyze results
    metrics = analyze_matches(matches_df, buyers_df, sellers_df)
    
    # Optionally filter for high confidence
    high_conf_df = matches_df[matches_df['confidence_score'] >= 0.95]
    output_high_conf = f'./matches/nlp_business_high_conf_{timestamp}.csv'
    high_conf_df.to_csv(output_high_conf, index=False)
    logging.info(f'Saved high confidence matches: {len(high_conf_df)} records => {output_high_conf}')
else:
    logging.info('No matches found.')

# Final memory cleanup
del seller_embeddings
gc.collect()


Another Semantic analysis

In [87]:
def match_locations(sales_locations, purchase_locations):
    """
    Check if any element of sales_locations is in purchase_locations.
    """
    if(any(loc in purchase_locations for loc in sales_locations)): print(f'purchase_locations: {purchase_locations}, sales_locations: {sales_locations}, {any(loc in purchase_locations for loc in sales_locations)}')
    
    return any(loc in purchase_locations for loc in sales_locations)

# Load updated datasets
logging.info('Loading datasets...')
buyers_df = pd.read_csv('./data/dejuna_buyer_latest_hf_nace_geocoded.csv')
sellers_df = pd.read_csv('./data/branche_nexxt_change_sales_listings_scrape_hf_nace_geocoded.csv')

# Preprocess buyers' text fields
logging.info('Preprocessing buyers\' text fields...')
buyers_df['title_preprocessed'] = buyers_df['title'].apply(lambda x:  preprocess_text(x, nlp))
buyers_df['description_preprocessed'] = buyers_df['description'].apply(lambda x:  preprocess_text(x, nlp))
buyers_df['long_description_preprocessed'] = buyers_df['long_description'].apply(lambda x:  preprocess_text(x, nlp))
buyers_df['preprocessed_branchen'] = buyers_df.apply(
    lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
)

# Preprocess sellers' text fields
logging.info('Preprocessing sellers\' text fields...')
sellers_df['title_preprocessed'] = sellers_df['title'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['description_preprocessed'] = sellers_df['description'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['long_description_preprocessed'] = sellers_df['long_description'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['branchen_preprocessed'] = sellers_df['branchen'].apply(lambda x:  preprocess_text(x, nlp))


#Location processing
logging.info('Processing locations...')
sellers_df['processed_location'] = sellers_df['location'].apply(lambda x: list(_extract_location_parts(x)) if pd.notnull(x) else [])
buyers_df['processed_location'] = buyers_df['location'].apply(lambda x: list(_extract_location_parts(x)) if pd.notnull(x) else [])

# Parse latitude and longitude from JSON strings
sellers_df['latitude'] = sellers_df['latitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])
sellers_df['longitude'] = sellers_df['longitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])

buyers_df['latitude'] = buyers_df['latitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])
buyers_df['longitude'] = buyers_df['longitude'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])


# Combine text fields
logging.info('Combining text fields...')
buyers_df['combined_text'] = buyers_df.apply(combine_text_fields, axis=1)
sellers_df['combined_text'] = sellers_df.apply(combine_text_fields, axis=1)


# # Augment text with synonyms
# logging.info('Augmenting buyers\' text with synonyms...')
# buyers_df['augmented_text'] = buyers_df['combined_text'].apply(lambda x: augment_text_with_synonyms(x, top_n=5))
# logging.info('Augmenting sellers\' text with synonyms...')
# sellers_df['augmented_text'] = sellers_df['combined_text'].apply(lambda x: augment_text_with_synonyms(x, top_n=5))

# # Preprocess augmented text
# buyers_df['final_text'] = buyers_df['combined_text'].apply(preprocess_text_de) + ' ' + buyers_df['augmented_text'].apply(preprocess_text_de)
# sellers_df['final_text'] = sellers_df['combined_text'].apply(preprocess_text_de) + ' ' + sellers_df['augmented_text'].apply(preprocess_text_de)

# -----------------------------------
# 4. Load Models
# -----------------------------------
logging.info("Loading SentenceTransformer and CrossEncoder models...")
# bi_encoder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
# bi_encoder = SentenceTransformer("xlm-roberta-base")
bi_encoder = SentenceTransformer('all-MiniLM-L12-v2')
cross_encoder = CrossEncoder('cross-encoder/stsb-roberta-large')

# -----------------------------------
# 5. Encode Sellers' Text
# -----------------------------------
logging.info("Encoding sellers' text...")
seller_texts = sellers_df['combined_text'].tolist()
seller_embeddings = bi_encoder.encode(seller_texts, convert_to_numpy=True, normalize_embeddings=True)
logging.info("Sellers' embeddings generated.")

# -----------------------------------
# 6. Matching Parameters
# -----------------------------------
similarity_threshold = 0.60
cross_encoder_threshold = 0.65
top_n = 100  # Number of top candidates to re-rank per buyer

matches = []
confidence_scores = []

total_buyers = len(buyers_df)
logging.info("Starting matching process...")

# -----------------------------------
# 7. Matching Loop
# -----------------------------------
matched_locations = []

for i, buyer_row in buyers_df.iterrows():
    buyer_text = buyer_row['combined_text']
    buyer_latitudes = buyers_df.iloc[i]['latitude']
    buyer_longitudes = buyers_df.iloc[i]['longitude']
    buyer_locations = buyers_df.iloc[i]['processed_location']


    # Encode buyer text
    buyer_embedding = bi_encoder.encode([buyer_text], convert_to_numpy=True, normalize_embeddings=True)
    
    # Compute cosine similarities with all sellers
    sim_scores = cosine_similarity(buyer_embedding, seller_embeddings)[0]
    
    # Get indices of sellers with similarity >= threshold
    matching_indices = np.where(sim_scores >= similarity_threshold)[0]
    
    if len(matching_indices) == 0:
        continue  # No matches above threshold
    
    # Select top N matches based on similarity scores
    top_indices = matching_indices[np.argsort(sim_scores[matching_indices])[::-1][:top_n]]
    
    # Prepare pairs for cross-encoder
    buyer_texts = [buyer_text] * len(top_indices)
    seller_texts_top = [sellers_df.iloc[idx]['combined_text'] for idx in top_indices]
    pairs = list(zip(buyer_texts, seller_texts_top))
    
    # Get cross-encoder scores
    cross_scores = cross_encoder.predict(pairs)
    
    # Filter based on cross-encoder threshold
    for seller_idx, cross_score in zip(top_indices, cross_scores):


        if cross_score >= cross_encoder_threshold:
            seller_row = sellers_df.iloc[seller_idx]
            seller_latitudes = sellers_df.iloc[seller_idx]['latitude']
            seller_longitudes = sellers_df.iloc[seller_idx]['longitude']
            seller_locations = sellers_df.iloc[seller_idx]['processed_location']

            location_match = False
            distance_km = None

            if match_locations(seller_locations, buyer_locations):
                location_match = True
            # else:
            #     # Calculate distance between all combinations of seller and buyer coordinates
            #     for s_lat, s_lon in zip(seller_latitudes, seller_longitudes):
            #         for p_lat, p_lon in zip(buyer_latitudes, buyer_longitudes):
            #             if None in [s_lat, s_lon, p_lat, p_lon]:
            #                 continue
            #             distance = geodesic((s_lat, s_lon), (p_lat, p_lon)).km
            #             if distance <= 50:  # 50 km threshold
            #                 distance_km = distance
            #                 location_match = True
            #                 break
            #         if location_match:
            #             break
            # if location_match:

                match = {
                    'buyer_id': buyer_row.get('id', ''),
                    'buyer_date': buyer_row.get('date', ''),
                    'buyer_title': buyer_row.get('title', ''),
                    'buyer_description': buyer_row.get('description', ''),
                    'buyer_long_description': buyer_row.get('long_description', ''),
                    'buyer_location': buyer_row.get('location', ''),
                    'buyer_nace_code': buyer_row.get('nace_code', ''), 

                    'seller_id': seller_row.get('id', ''),
                    'seller_date': seller_row.get('date', ''),
                    'seller_title': seller_row.get('title', ''),
                    'seller_description': seller_row.get('description', ''),
                    'seller_long_description': seller_row.get('long_description', ''),
                    'seller_location': seller_row.get('location', ''),
                    'seller_source': seller_row.get('url', ''),
                    'seller_nace_code': seller_row.get('nace_code', ''),
                    
                    'similarity_score': cross_score,
                    'distance_km': distance_km if distance_km else 'Within processed locations'

                }
                matches.append(match)
                confidence_scores.append(cross_score)
    
    # Progress Logging
    if (i + 1) % 50 == 0:
        logging.info(f"Processed {i+1}/{total_buyers} buyers.")

# -----------------------------------
# 8. Create Matches DataFrame
# -----------------------------------
logging.info("Creating matches DataFrame...")
matches_df = pd.DataFrame(matches)
        
if not matches_df.empty:
    matches_df['confidence_score'] = confidence_scores
    matches_df = matches_df.sort_values('confidence_score', ascending=False)
    
    # Save All Matches
    timestamp = datetime.now().strftime("%d_%H-%M")
    output_all = f'./matches/nlp_business_all_matches_{timestamp}.csv'
    matches_df.to_csv(output_all, index=False)
    matches_df.to_excel(f'./matches/nlp_business_all_matches_{timestamp}.xlsx', index=False)
    logging.info(f"Saved all matches: {len(matches_df)} records => {output_all}")
    
    # Save High Confidence Matches
    high_conf_df = matches_df[matches_df['confidence_score'] >= cross_encoder_threshold]
    output_high_conf = f'./matches/nlp_business_high_conf_{timestamp}.csv'
    high_conf_df.to_csv(output_high_conf, index=False)
    logging.info(f"Saved high confidence matches: {len(high_conf_df)} => {output_high_conf}")
else:
    logging.info("No matches found.")

# -----------------------------------
# 9. Memory Cleanup
# -----------------------------------
del seller_embeddings
gc.collect()


2025-01-21 12:42:44,324 - INFO - Loading datasets...
2025-01-21 12:42:44,447 - INFO - Preprocessing buyers' text fields...
2025-01-21 12:42:45,212 - INFO - Preprocessing sellers' text fields...


In [43]:
# Read sellers and buyers files into dataframes
sellers_df = pd.read_csv(sellers_filepath)
buyers_df = pd.read_csv(buyer_filepath)

# Add origin column to both dataframes
sellers_df['origin'] = 'seller'
buyers_df['origin'] = 'buyer'
buyers_df['branchen'] = buyers_df.apply(
    lambda row: f"{row['Industrie']} > {row['Sub-Industrie']}", axis=1
)


# Concatenate both dataframes
combined_df = pd.concat([sellers_df, buyers_df], ignore_index=True)

# Display the first few rows of the combined dataframe

5256                          Handwerk > Gebäudereinigung
5257                          Immobilien > Hausverwaltung
5258    Handwerk > Bestattungsgewerbe, Bestattungsunte...
5259                          Immobilien > Hausverwaltung
5260             Handwerk > Heizung, Sanitär, Klima (SHK)
5261                          Immobilien > Hausverwaltung
5262                          Immobilien > Hausverwaltung
5263    Handwerk > Bestattungsgewerbe, Bestattungsunte...
5264    Gesundheits- und Sozialwesen > Gesundheitswese...
5265    Gesundheits- und Sozialwesen > Gesundheitswese...
Name: branchen, dtype: object


NACE Labels for Combined data

In [51]:

# -------------------------------------------------------------------------
# 5. Load your Seller/Branchen data and NACE codes
# -------------------------------------------------------------------------
def load_data(nace_codes_filepath):
    nace_codes = load_nace_codes(nace_codes_filepath)
    return nace_codes

# -------------------------------------------------------------------------
# 6. MAIN LOGIC
# -------------------------------------------------------------------------
    # Filepaths (update to your actual paths)
# sellers_filepath = originalSalesNexxtChangeData       # CSV with a column 'branchen'
sellers_filepath =  dejunaPurchases    # CSV with a column 'branchen'
nace_codes_filepath = nacecode_array_obj_du

# Load data
sellers_df = combined_df 
nace_codes = load_data(nace_codes_filepath)
print("🚀 Sellers and NACE codes loaded.")

# Initialize the Sentence Transformer model
# For German or multilingual, consider e.g.: 
#    model_name = 'paraphrase-multilingual-mpnet-base-v2'
model_name = 'all-MiniLM-L12-v2'
model = SentenceTransformer(model_name)
print(f"🚀 Loaded SentenceTransformer model: {model_name}")

# ---------------------------------------------------------------------
# 6a. Preprocess NACE descriptions
# ---------------------------------------------------------------------
# Convert each NACE code's description with the same text preprocessing
nace_descriptions = [preprocess_text(desc, nlp) for desc in nace_codes.values()]
# Create embeddings for these descriptions
nace_embeddings = get_embedding_batch(nace_descriptions, model)
# We'll keep a list of NACE codes in the same order
nace_code_list = list(nace_codes.keys())
print("🚀 Created embeddings for NACE descriptions.")

# ---------------------------------------------------------------------
# 6b. Preprocess 'branchen' column in sellers data
# ---------------------------------------------------------------------
# We'll store it in a new column 'preprocessed_branchen'
# sellers_df['preprocessed_branchen'] = sellers_df['branchen'].apply(lambda x: preprocess_text(x, nlp))
# Split 'branchen' on '>' and join each part


logging.info('Preprocessing buyers\' text fields...')
sellers_df['title_preprocessed'] = sellers_df['title'].apply(lambda x:  preprocess_text(x, nlp))
sellers_df['description_preprocessed'] = sellers_df['description'].apply(lambda x:  preprocess_text(x, nlp))
# sellers_df['long_description_preprocessed'] = sellers_df['long_description'].apply(lambda x:  preprocess_text(x, nlp))
# sellers_df['preprocessed_branchen'] = sellers_df.apply(
#     lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
# )

sellers_df['preprocessed_branchen'] = sellers_df['branchen'].apply(
    lambda x: ' '.join(x.split('>'))
)

def combine_text_fields(row):
    return ' '.join([
        row.get('title_preprocessed', ''),
        row.get('description_preprocessed', ''),
        # row.get('long_description_preprocessed', ''),
        row.get('branchen_preprocessed', '')
    ])
sellers_df['combined_text'] = sellers_df.apply(combine_text_fields, axis=1)

# # Join 'Sub-Industrie' and 'Industrie' columns for buyer data
# sellers_df['preprocessed_branchen'] = sellers_df.apply(
#     lambda row: f"{row['Industrie']} {row['Sub-Industrie']}", axis=1
# )

# Create embeddings for all sellers' branchen
branchen_embeddings = get_embedding_batch(sellers_df['combined_text'].tolist(), model)
print("🚀 Created embeddings for sellers' 'branchen' field.")

# ---------------------------------------------------------------------
# 6c. Compute similarity and assign best-match NACE code
# ---------------------------------------------------------------------
similarities = cosine_similarity(branchen_embeddings, nace_embeddings)
best_match_indices = similarities.argmax(axis=1)

# For each seller row, pick the NACE code with highest similarity
sellers_df['assigned_nace_code'] = [nace_code_list[idx] for idx in best_match_indices]
sellers_df['assigned_nace_similarity'] = [similarities[i][idx] for i, idx in enumerate(best_match_indices)]
print("🚀 Assigned preliminary NACE codes based on 'branchen' similarity.")

# Optionally set a threshold. If similarity < threshold => 'Unassigned'
similarity_threshold = 0.4
sellers_df['nace_code'] = sellers_df.apply(
    lambda row: row['assigned_nace_code'] 
                if row['assigned_nace_similarity'] >= similarity_threshold 
                else 'Unassigned',
    axis=1
)

# ---------------------------------------------------------------------
# 6d. Save and review
# ---------------------------------------------------------------------
output_file = "buyers_sellers_combined_nace.csv"
sellers_df.to_csv(output_file, index=False)
print(f"🚀 Saved sellers data with assigned NACE codes to: {output_file}\n")

# Print a small sample
print("Sample of NACE Code Assignments:")
print(sellers_df[['assigned_nace_code', 'assigned_nace_similarity', 'nace_code']].head(10))


2025-01-16 20:04:56,995 - INFO - Use pytorch device_name: mps
2025-01-16 20:04:56,996 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L12-v2


🚀 Sellers and NACE codes loaded.
🚀 Loaded SentenceTransformer model: all-MiniLM-L12-v2


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2025-01-16 20:05:04,151 - INFO - Preprocessing buyers' text fields...


🚀 Created embeddings for NACE descriptions.


Batches:   0%|          | 0/83 [00:00<?, ?it/s]

🚀 Created embeddings for sellers' 'branchen' field.
🚀 Assigned preliminary NACE codes based on 'branchen' similarity.
🚀 Saved sellers data with assigned NACE codes to: buyers_sellers_combined_nace.csv

Sample of NACE Code Assignments:
  assigned_nace_code  assigned_nace_similarity   nace_code
0              C24.1                  0.384563  Unassigned
1                I56                  0.593983         I56
2            O84.1.3                  0.508671     O84.1.3
3                M73                  0.521514         M73
4              C26.6                  0.583879       C26.6
5              C23.7                  0.386995  Unassigned
6                M73                  0.455993         M73
7              G47.5                  0.439482       G47.5
8              M70.1                  0.376641  Unassigned
9                K65                  0.449326         K65


In [58]:
temp1= pd.read_csv("./matches/nlp_business_all_matches_16_19-31.csv")
temp2= pd.read_csv("./matches/nlp_business_all_matches_15_13-11.csv")

# Concatenate the dataframes
combined_df = pd.concat([temp1, temp2])
# Get all duplicates based on specific columns
# Get all rows that only exist once in the dataframe
unique_rows = combined_df.drop_duplicates(subset=['buyer_date', 'buyer_title', 'buyer_description', 'buyer_location', 'seller_date', 'seller_title', 'seller_description', 'seller_location'], keep=False)
duplicates = unique_rows[unique_rows.duplicated(subset=['buyer_date', 'buyer_title', 'buyer_description', 'buyer_location', 'seller_date', 'seller_title', 'seller_description', 'seller_location'], keep=False)]

# unique_rows.to_csv('./matches/unique_rows.csv', index=False)
# Display the unique rows
# print(unique_rows)
print(duplicates)
# Display the duplicates
# print(duplicates)

Empty DataFrame
Columns: [buyer_id, buyer_date, buyer_title, buyer_description, buyer_long_description, buyer_location, buyer_nace_code, seller_id, seller_date, seller_title, seller_description, seller_long_description, seller_location, seller_source, seller_nace_code, similarity_score, distance_km, confidence_score]
Index: []


In [65]:
from geopy.distance import geodesic

def extract_locations(location_string):
    """
    Extracts locations from a string with nested structure.
    
    Args:
        location_string (str): Input string containing locations in nested format.

    Returns:
        list: A list of extracted locations.
    """
    locations = []

    # Split the input into lines
    lines = location_string.splitlines()

    for line in lines:
        # Strip whitespace from the line
        line = line.strip()
        
        # Ignore empty lines
        if not line:
            continue

        # If the line contains '>', it indicates nested locations
        if '>' in line:
            # Extract the specific location after the last '>'
            nested_location = line.split('>')[-1].strip()
            locations.append(nested_location)
        else:
            # Append the standalone location
            locations.append(line)

    return locations

def get_location_coordinates(location):
    """
    Mock function to return coordinates for a given location.
    Replace this with an actual geocoding API in production.

    Args:
        location (str): The location name.

    Returns:
        tuple: Latitude and longitude of the location.
    """
    # Mock coordinates for demonstration purposes
    coordinates = {
        "Celle": (52.6226, 10.0815),
        "Hannover": (52.3759, 9.7320),
        "Karlsruhe": (49.0069, 8.4037),
        "Mannheim": (49.4875, 8.4660),
        "Goettingen": (51.5413, 9.9158),
    }
    return coordinates.get(location, None)

def locations_match(s1, s2):
    """
    Checks if the locations extracted from two strings have any matches.
    If no exact match, it calculates the distance between locations.

    Args:
        s1 (str): First input string containing locations.
        s2 (str): Second input string containing locations.

    Returns:
        bool: True if there are matching locations or distance < 50KM, False otherwise.
    """
    # Extract locations from both strings
    locations1 = set(_extract_location_parts(s1))
    locations2 = set(_extract_location_parts(s2))

    print(f"Locations 1: {locations1}")
    print(f"Locations 2: {locations2}")
    # Check for exact match first
    if not locations1.isdisjoint(locations2):
        return True

    # If no exact match, calculate distances
    for loc1 in locations1:
        for loc2 in locations2:
            coord1 = get_location_coordinates(loc1)
            coord2 = get_location_coordinates(loc2)

            if coord1 and coord2:
                distance = geodesic(coord1, coord2).kilometers
                print(f"Distance between {loc1} and {loc2}: {distance} KM")
                if distance < 50:
                    return True

    return False

# Example usage
s1 = "Niedersachen > Celle"
s2 = "Niedersachsen > Goettingen"

output = locations_match(s1, s2)
print(output)

niedersachen
celle
niedersachsen
goettingen
Locations 1: {'Celle', 'Niedersachen'}
Locations 2: {'Goettingen', 'Niedersachsen'}
Distance between Celle and Goettingen: 120.84999834028793 KM
False
